In [ ]:
from datasets import Dataset, DatasetDict
from datasets import load_dataset
from tqdm import tqdm
import pandas as pd
dataset = load_dataset('ibm/duorc', 'SelfRC')

In [ ]:
def process_split(split_name):
    keep_indices = []
    for idx, row in tqdm(enumerate(dataset[split_name])):
        flag = 1
        if row['no_answer'] == False:
            for ans in row['answers']:
                if ans not in row['plot']:
                    flag = 0
                    break
            if flag == 1:
                keep_indices.append(idx)
        else:
            keep_indices.append(idx)
    
    dataset_subset = dataset[split_name].select(keep_indices)
    dataset_df = pd.DataFrame(dataset_subset)
    
    answer_dicts = []
    for row in tqdm(dataset_df.itertuples(index=False)):
        answer_idxs = []
        if row.no_answer == False:
            for ans in row.answers:
                answer_idxs.append(row.plot.find(ans))
            answer_dicts.append({'text': row.answers, 'answer_start': answer_idxs})
        else:
            answer_dicts.append({'text': [], 'answer_start': []})
    
    dataset_df.drop(columns = ['answers'], inplace=True)
    dataset_df['answers'] = answer_dicts
    
    return dataset_df

In [ ]:
train_hf = Dataset.from_pandas(process_split('train'))
validation_hf = Dataset.from_pandas(process_split('validation')) 
test_hf = Dataset.from_pandas(process_split('test'))

In [ ]:
processed_dataset = DatasetDict()

processed_dataset['train'] = train_hf
processed_dataset['validation'] = validation_hf
processed_dataset['test'] = test_hf

In [ ]:
processed_dataset = processed_dataset.rename_column("plot", "context")

In [ ]:
processed_dataset.push_to_hub('Saptarshi7/duorc_processed', private=True)